## **Оглавление**

### **[Введение](#1)**
### **1. [Обзор данных](#2)**  
### **2. [Предобработка данных](#3)**  
    2.1 Обработка пропусков
    2.2 Обработка некорректных значений
    2.3 Замена типа данных  
    2.4 Обработка дубликатов
    2.5 Лемматизация
    2.6 Категоризация данных
### **3. [Проверка гипотезы](#4)**  
### **[Итоги исследования](#5)**

# Исследование надёжности заёмщиков

## Введение  <a id='1'></a>
Заказчик — кредитный отдел банка. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  

**Цель исследования** — проверить гипотезу: семейное положение и количество детей клиента влияет на факт погашения кредита в срок.  

Исследование проводится в три этапа:

1. Обзор данных.
2. Предобработка данных.
3. Проверка гипотезы.

## Обзор данных <a id='2'></a>

Ознакомимся с входными данными.

In [52]:
#импортируем библиотеки
import pandas as pd

from pymystem3 import Mystem
m = Mystem()

In [53]:
data = pd.read_csv('/datasets/data.csv') #прочитаем файл с данными и сохраним его в переменной data
data.info() #получим общую информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [54]:
data.head() #просмотрим первые пять строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Согласно документации к данным:  

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов, где 0 - это отсутствие задолжности, а 1 - наличие задолженности.
- total_income — ежемесячный доход
- purpose — цель получения кредита


**Вывод**  

В таблице 12 столбцов, названия столбцов написаны корректно.

Количество non-null значений в столбцах различается, следовательно в столбцах days_employed и total_income есть пропуски. В колонках days_employed и total_income лучше поменять тип данных на int64, убрать дробную часть. Встречаются некорректные значения и дубликаты.  

В каждой строке таблицы — данные о клиенте банка. Колонки описывают возраст клиента, пол, его образование, тип занятости, трудовой стаж, ежемесячный доход, семейный статус и кол-во детей, а также цель получения кредита и наличие/отсутствие задолженности.

## Предобработка данных <a id='3'></a>

### Обработка пропусков

#### Посчитаем количество пропусков в каждом столбце

In [55]:
data.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков days_employed и total_income совпадает, возможно, это одни и те же строки. Количество строк с пропусками в каждом из этих столбцов составляет примерно 10% (21525/2174*100), поэтому просто удалить строку с недостающими данными мы не можем, это может повлиять на результаты исследования.  

В столбце days_employed, помимо большого количества пропусков, некоторые значения указаны некорректно, а также сами значения в формате, который трудно сопоставить с количеством дней. Так как эти данные не важны для исследования, столбец можно удалить.  

Пропуски в столбце total_income можно восстановить медианными значениями по срезам.

#### Удалим столбец days_employed

In [56]:
data = data.drop(['days_employed'], axis = 1) #удаление столбца days_employed
data.columns #выведем названия всех столбцов, чтобы увидеть, что столбец days_employed отсутствует

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

#### Определим причину пропусков в столбце total_income

- посмотрим, как пропуски распределены в столбце total_income
- сравним доли типов занятости в пропущенных значениях с долями типов занятости в исходной таблице

In [57]:
data_total_income = data[data['total_income'].isna()]
#найдем пропуски в столбце total_income и сохраним их в переменную data_total_income

data_total_income['income_type'].value_counts(normalize=True)
#найдем долю каждого типа занятости в общем количестве пропусков

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

In [58]:
data['income_type'].value_counts(normalize=True) #найдем доли каждого типа занятости в исходной таблице

сотрудник          0.516562
компаньон          0.236237
пенсионер          0.179141
госслужащий        0.067782
безработный        0.000093
предприниматель    0.000093
в декрете          0.000046
студент            0.000046
Name: income_type, dtype: float64

Результаты сравнения показывают, что больше всего пропусков встречается в строках с типом занятости "сотрудник". Характер распределения всех доль типов занятости в обоих случаях очень похож, поэтому мы можем сделать вывод, что пропуски в столбцах total_income, скорее всего, случайные.

#### Заменим пропуски в столбце total_income

In [59]:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type', 'education', 'gender'])['total_income'].transform('median'))
#заменим пропуски в столбце total_income на медианное значение в разрезе групп по типу занятости, образовению и полу.

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [60]:
data.isna().sum() #посчитаем сумму пропусков в каждом столбце

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        1
purpose             0
dtype: int64

Остался один пропуск. Посмотрим на эту строку.

In [61]:
data[data['total_income'].isna()] #выведем строку с пропуском

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости


Скорее всего, тип занятости "предпринематель" не удалось объединить в группы, поэтому не получилось заполнить пропуск.

In [62]:
data['income_type'].value_counts() #выведем все уникальные значения столбца total_income и количество их упоминаний.

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

"Предпринематель" встречается два раза. Посмотрим на эти строки.

In [63]:
data.loc[data.loc[:, 'income_type'] == 'предприниматель'] #выведем все строки, где тип занятости - "предприниматель"

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


Главные показатели для исследования children и family_status у обоих "предпринимателей" схожи. Заменим пропуск в доходе в строке 5936 на значение в строке 18697.

In [64]:
data['total_income'] = data['total_income'].fillna(499163.144947) #заменим пропущенное значение на доход из строки 18697

Убедимся, что замена произошла.

In [65]:
data.loc[data.loc[:, 'income_type'] == 'предприниматель'] #выведем все строки, где тип занятости - "предприниматель"

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,58,высшее,0,женат / замужем,0,M,предприниматель,0,499163.144947,покупка жилой недвижимости
18697,0,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


Выведем общую информацию о таблице, чтобы убедиться, что во всех столбцах пропуски отсутствуют.

In [66]:
data.info() #получаем общую информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


### Обработка некорректных значений в столбцах

Просмотрим уникальные значения во всех столбцах кроме income_type и total_income, чтобы найти некорректные значения.

In [67]:
unique_colomns = data.drop(['income_type', 'total_income'], axis=1)
#сохраним в переменную unique_colomns таблицу со всеми столбцами, кроме income_type и total_income
for col in unique_colomns: #создаем цикл, который выведет уникальные значения по всем столбцам
    print(col, ': ', data[col].unique())
    print()

children :  [ 1  0  3  2 -1  4 20  5]

dob_years :  [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

education :  ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

education_id :  [0 1 2 3 4]

family_status :  ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

family_status_id :  [0 1 2 3 4]

gender :  ['F' 'M' 'XNA']

debt :  [0 1]

purpose :  ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвиж

1) В столбце children встречаются такие значения как -1 и 20, которые нужно заменить на 1 и 2 соответственно.

In [68]:
data['children'] = data['children'].replace(-1,1) #заменяем -1 на 1 в столбце children
data['children'] = data['children'].replace(20,2) #заменяем 20 на 2 в столбце children

2) В столбце dob_years видим, что есть возраст равный 0. Выведем уникальные значения и количество их упоминаний, чтобы понять как часто 0 встречается в значениях.

In [69]:
data['dob_years'].value_counts() #выводим уникальные значения столбца dob_years и кол-во их упоминаний

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

0 встречается 101 раз, что составляет всего 0,5% (101/21525*100) от общего количества строк. При этом, возраст не является приоритетным критерием в определении платёжеспособности клиентов. Поэтому оставляем значения равные 0. Это не повлияет на результаты исследования.

3) В столбце gender есть значение 'XNA', которое лучше заменить на понятное 'unknow'.

In [70]:
data['gender'] = data['gender'].replace('XNA', 'unknow') #заменим значение 'XNA' на 'unknow'

4) В столбце family_status приведем написание 'Не женат / не замужем' к единому стилю. Приведем все строки в столбце family_status к нижнему регистру.

In [71]:
data['family_status'] = data['family_status'].str.lower() #приведем строки к нижнему регистру

В остальных столбцах все значения корректны.  
Снова выведем уникальные значения, чтобы удостовериться, что данные были изменены.

In [72]:
for col in unique_colomns: #снова применим цикл, который выведет уникальные значения по всем столбцам
    print(col, ': ', data[col].unique())
    print()

children :  [1 0 3 2 4 5]

dob_years :  [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

education :  ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

education_id :  [0 1 2 3 4]

family_status :  ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']

family_status_id :  [0 1 2 3 4]

gender :  ['F' 'M' 'unknow']

debt :  [0 1]

purpose :  ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 

**Вывод**  
В столбцах  days_employed и total_income были найдены пропуски. Столбец days_employed был удален из-за большого количества пропусков и некорректных значений, отсутствие данных о стаже не повлияет на результат исследования.

Пропуски в столбце total_income имеют случайный характер, поэтому мы заменили их на медианные значения по группам людей с одинаковым типом занятости, одного пола и уровня образования.  

В столбцах children и gender привели данные к корректным значениям.  

В столбце dob_years оставили значения равные 0, их не много и они не помешают исследованию.  

Строки в столбце family_status были приведены к единому стилю написания. 

### Замена типа данных

Посмотрим, каким типам соответствуют данные в столбцах.

In [73]:
data.dtypes #выведем информацию по типам данных по каждому столбцу

children              int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Столбец total_income имеет тип данных float64, изменим тип данных на Int64 и округлим значения для удобства работы с данными. Так как пропусков в столбце нет, используем метод astype().

In [74]:
data['total_income'] = data['total_income'].round().astype('Int64')
#в столбце total_income убираем дробную часть и меняем тип данных на int64.

Убедимся, что замена произошла.

In [75]:
data.head()  #выведем первые 5 строк таблицы

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [76]:
data.dtypes #выведем информацию по типам данных по каждому столбцу

children             int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         Int64
purpose             object
dtype: object

**Вывод**  
Привели значения в столбце total_income к типу  Int64 и убрали дробную часть.

### Обработка дубликатов

#### Найдем неявные дубликаты

Найдем и исправим неявные дубликаты для столбца education.

In [77]:
data['education'].unique() #выведем уникальные значения в столбце education

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведем все строки в столбце education к нижнему регистру и проверим выполнение.

In [78]:
data['education'] = data['education'].str.lower() #приводим строки к нижнему регистру

In [79]:
data['education'].unique() #выводим все уникальные значения

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

#### Найдем явные дубликаты

Найдем и удалим явные дубликаты.

In [80]:
data.duplicated().sum() #находим кол-во явных дубликатов

54

In [81]:
data = data.drop_duplicates().reset_index(drop=True)
#удаляем дубликаты, одновременно создаем новые индексы и удаляем старые

In [82]:
data.duplicated().sum() #проверяем кол-во дубликатов после удаления

0

**Вывод**  
Неявные дубликаты в данных - написание одинаковых слов в разном регистре в столбце education - были исправлены. Удалены 54 строки явных дубликатов. На появление неявных дубликатов влияет человеческий фактор. Полные дубликаты могут быть причиной технической ошибки базы данных.

### Лемматизация

Можно заметить, что цели получения кредита схожи, но написаны по разному.  
Просмотрим уникальные значения в столбце purpose, чтобы узнать цели для получения кредита. Для удобства отсортируем по алфавиту.

In [83]:
sorted(data['purpose'].unique()) #выводим все уникальные значения, сортируя их по алфавиту

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

Найдем леммы слов в строках, записанных в разных формах, и создадим новый столбец, в котором раскидаем похожие цели на 4 категории: недвижемость, автомобиль, образование и свадьба.

In [84]:
def new_category_purpose(row):
#создадим функцию, которая будет искать заданные леммы в строках и возвращать название нужной категории
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if 'образование' in lem_purpose:
        return 'образование'
    if 'жилье' in lem_purpose or 'недвижимость' in lem_purpose:
        return 'недвижимость'
    if 'свадьба' in lem_purpose:
        return 'свадьба'

data['purpose_category'] = data.apply(new_category_purpose, axis=1) #добавим новый столбец с категориями

In [85]:
data.head() #выведем первые 5 строк таблицы, чтобы проверить выполнение предыдущего действия

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**  
Добавили новый столбец с категориями недвижимость, автомобиль, образование и свадьба, созданный на основе схожих целей получения кредита.

### Категоризация данных

Для удобства работы с данными поделим значения в столбцах total_income и children на группы. 

#### Деление на группы столбца total_income

Столбец total_income разделим на 5 групп в зависимости от величины дохода, для удобства работы с данными присвоим каждой группе название: "низкий", "ниже среднего", "средний", "выше среднего", "высокий".

In [86]:
bin_labels_5 = ["низкий", "ниже среднего", "средний", "выше среднего", "высокий"] #создадим список с названиями интервалов

data['total_income_group'] = pd.qcut(data['total_income'], q=[0, .2, .4, .6, .8, 1], labels=bin_labels_5)
#создадим новый столбец total_income_group: разделим все значения столбца total_income на 5 интервалов
#определим диапазон используемых квантилей и присвоим группам названия

data.head() #выведем первые 5 строк таблицы

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,недвижимость,высокий
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,ниже среднего
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,недвижимость,средний
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,образование,высокий
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний


In [87]:
# КОД РЕВЬЮЕРА

pd.qcut(data['total_income'], 5, ['низкий','ниже среднего','средний','выше среднего', 'высокий'])

0              высокий
1        ниже среднего
2              средний
3              высокий
4              средний
             ...      
21466          высокий
21467          средний
21468           низкий
21469          высокий
21470           низкий
Name: total_income, Length: 21471, dtype: category
Categories (5, object): [низкий < ниже среднего < средний < выше среднего < высокий]

#### Деление на группы столбца children

В зависимости от количества детей столбец children разделим на 3 группы: "бездетные", "1 или 2 ребенка", "многодетные".

In [88]:
def category_children(children): 
#создадим функцию, которая будет сравнивать кол-во детей в каждой строке, и присваивать нужное название группы
    if children == 0:
        return 'бездетные'
    if children == 1 or children == 2:
        return '1 или 2 ребенка'
    if children > 2:
        return 'многодетные'  

In [89]:
data.insert(1, 'children_group', data['children'].apply(category_children))
#добавим новый столбец total_income_group с названиями групп рядом со столбцом children 

data.head() #выводим первые 5 строк таблицы

,children,children_group,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_group
0,1,1 или 2 ребенка,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,недвижимость,высокий
1,1,1 или 2 ребенка,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,ниже среднего
2,0,бездетные,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,недвижимость,средний
3,3,многодетные,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,образование,высокий
4,0,бездетные,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний


**Вывод**  
Разделили данные в столбцах total_income и children на группы в зависимости от величины ежемесячного дохода и кол-ва детей соответственно.

## Проверка гипотезы <a id='4'></a>

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Чтобы определить, есть ли зависимоть между наличием детей и возвратом кредита в срок, создадим сводную таблицу, которую сгруппируем по количеству детей, и найдем наличие/отсутствие задолжностей по кредиту в каждой группе. В таблице зададим последовательность названий групп в нужном нам порядке: сначала "бездетные", потом "1 или 2 ребенка" и "многодетные".

In [90]:
data['children_group'] = pd.Categorical(data['children_group'], ordered=True, categories=['бездетные','1 или 2 ребенка','многодетные'])
#зададим порядок вывода названий групп по кол-ву детей

children_debt_pivot = data.pivot_table(index=['children_group'], columns='debt', values='total_income', aggfunc='count')
#выведем сводную таблицу по кол-ву детей и кол-ву возврата/невозврата кредита и сохраняем ее в переменную children_debt_pivot

children_debt_pivot #выведем получившуюся таблицу

debt,0,1
children_group,,
бездетные,13044,1063
1 или 2 ребенка,6337,647
многодетные,349,31


Создадим новый столбец percent, в который запишем процент возврата кредита в каждой группе. Для этого разделим кол-во возвратов в срок на общее кол-во кредитов для каждой группы.

In [91]:
children_debt_pivot['percent'] = children_debt_pivot[0] / (children_debt_pivot[1] + children_debt_pivot[0]) * 100
#создадим новый столбец percent, в который запишем процент возврата кредитов в каждой группе

children_debt_pivot.round(2) #округлим значения до двух знаков после запятой

debt,0,1,percent
children_group,,,
бездетные,13044,1063,92.46
1 или 2 ребенка,6337,647,90.74
многодетные,349,31,91.84


**Вывод**   
Из таблицы можно сделать вывод, что самый высокий процент возврата в бездетной группе. Самый низкий у группы с 1 или 2 детьми. Сильного разброса между группами не наблюдается.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Создадим сводную таблицу, которую сгруппируем по семейному положению, и найдем наличие/отсутствие задолженностей по кредиту в каждой группе. Также в таблице зададим последовательность названий групп в нужном нам порядке.

In [92]:
data['family_status'] = pd.Categorical(data['family_status'], ordered=True, categories=['не женат / не замужем','гражданский брак','женат / замужем', 'в разводе', 'вдовец / вдова'])
#зададим порядок вывода названий групп по семейному статусу

family_status_debt_pivot = data.pivot_table(index=['family_status'], columns='debt', values='total_income', aggfunc='count')
#выведем сводную таблицу по семейному положению и кол-ву возврата/невозврата кредита и сохраняем ее в переменную family_status_debt_pivot

family_status_debt_pivot #выведем получившуюся таблицу

debt,0,1
family_status,,
не женат / не замужем,2536,274
гражданский брак,3775,388
женат / замужем,11413,931
в разводе,1110,85
вдовец / вдова,896,63


Создадим новый столбец percent, в который запишем процент возврата кредита в каждой группе. Для этого разделим кол-во возвратов в срок на общее кол-во кредитов для каждой группы.

In [93]:
family_status_debt_pivot['percent'] = family_status_debt_pivot[0] / (family_status_debt_pivot[1] + family_status_debt_pivot[0]) * 100
#создадим новый столбец percent, в который запишем процент возврата кредитов в каждой группе

family_status_debt_pivot.round(2) #округлим значения до двух знаков после запятой

debt,0,1,percent
family_status,,,
не женат / не замужем,2536,274,90.25
гражданский брак,3775,388,90.68
женат / замужем,11413,931,92.46
в разводе,1110,85,92.89
вдовец / вдова,896,63,93.43


**Вывод**  
Самый высокий процент возврата у вдовцов/вдов, самый низкий процент возврата у незамужних/неженатых и в гражданском браке.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Создадим сводную таблицу, которую сгруппируем по уровню дохода и найдем наличие/отсутствие задолжностей по кредиту в каждой группе. Зададим последовательность названий групп в нужном нам порядке.

In [94]:
data['total_income_group'] = pd.Categorical(data['total_income_group'], ordered=True, categories=['низкий','ниже среднего','средний', 'выше среднего', 'высокий'])
#зададим порядок вывода названий групп по уровню дохода

total_income_debt_pivot = data.pivot_table(index=['total_income_group'], columns='debt', values='total_income', aggfunc='count')
#выведем сводную таблицу по уровню дохода и кол-ву возврата/невозврата кредита и сохраняем ее в переменную total_income_debt_pivot

total_income_debt_pivot #выведем получившуюся таблицу

debt,0,1
total_income_group,,
низкий,3951,344
ниже среднего,3938,356
средний,3917,377
выше среднего,3930,364
высокий,3994,300


Создадим новый столбец percent, в который запишем процент возврата кредита в каждой группе. Для этого разделим кол-во возвратов в срок на общее кол-во кредитов для каждой группы.

In [95]:
total_income_debt_pivot['percent'] = total_income_debt_pivot[0] / (total_income_debt_pivot[1] + total_income_debt_pivot[0]) * 100
#создадим новый столбец percent, в который запишем процент возврата кредитов в каждой группе

total_income_debt_pivot.round(2) #округлим значения до двух знаков после запятой

debt,0,1,percent
total_income_group,,,
низкий,3951,344,91.99
ниже среднего,3938,356,91.71
средний,3917,377,91.22
выше среднего,3930,364,91.52
высокий,3994,300,93.01


**Вывод**  
Самый низкий процент возврата по кредиту у людей со средним доходом, самый высокий у людей с высоким уровнем дохода.

- **Как разные цели кредита влияют на его возврат в срок?**

Создадим сводную таблицу, которую сгруппируем по целям кредита и найдем наличие/отсутствие задолжностей по кредиту в каждой группе.

In [96]:
purpose_category_debt_pivot = data.pivot_table(index=['purpose_category'], columns='debt', values='total_income', aggfunc='count')
#выведем сводную таблицу по уровню дохода и кол-ву возврата/невозврата кредита и сохраняем ее в переменную purpose_category_debt_pivot

purpose_category_debt_pivot #выведем получившуюся таблицу

debt,0,1
purpose_category,,
автомобиль,3905,403
недвижимость,10032,782
образование,3644,370
свадьба,2149,186


Создадим новый столбец percent, в который запишем процент возврата кредита в каждой группе. Для этого разделим кол-во возвратов в срок на общее кол-во кредитов для каждой группы.

In [97]:
purpose_category_debt_pivot['percent'] = purpose_category_debt_pivot[0] / (purpose_category_debt_pivot[1] + purpose_category_debt_pivot[0]) * 100
#создадим новый столбец, в который запишем процент возврата кредитов в каждой группе

purpose_category_debt_pivot.round(2) #округлим значения до двух знаков после запятой

debt,0,1,percent
purpose_category,,,
автомобиль,3905,403,90.65
недвижимость,10032,782,92.77
образование,3644,370,90.78
свадьба,2149,186,92.03


**Вывод**  
Видим, что самые высокие показатели возврата по кредиту на недвижимость и свадьбу. Самый низкий процент возврата по кредиту на автомобиль.

## Итоги исследования <a id='5'></a>

Прежде чем проверить гипотезу мы ознакомились с входными данными, провели их предобработку: заменили пропуски на подходящие значения, изменили тип данных, исправили некорректные значения, удалили явные и неявные дубликаты.  

В ходе исследования, отвечая на вопросы о зависимости возврата кредита в срок от наличия детей, семейного положения, уровня дохода и целей кредита, установили, что:  

- самые высокие показатели возврата кредита у вдовцов/вдов и людей в гражданском браке, а самые низкие показатели у незамужних/неженатых, разница между этими группами составляет 3%, что является значительным показателем для банка. Следовательно можно утверждать, что семейное положение напрямую влияет на погашение кредита в срок;
- показатели по кол-ву детей не так сильно разнятся, но есть увеличение процента возврата кредита в сторону бездетных, поэтому мы можем утверждать, что кол-во детей лишь частично влияет на погашение кредита;
- на возврат кредита также влияют его цели: категории "недвижемость" и "свадьба" имеют на 1,5-2% выше уровень возврата по кредиту в сравнении с другими категориями;
- по уровню дохода возврат по кредиту колеблется в пределах одного процента, немного выше процент возврата у людей с высоким доходом, но критической зависимости не наблюдается.  

Таким образом гипотеза подтверждена частично: семейное положение влияет на погашение кредита в срок, а кол-во детей не имеет критического значения.  
Также в ходе исследования, были обнаружены, что цели кредита в значительной степени влияют на возврат кредита в срок.
